In [53]:
#B.2.a
from uncertainties import ufloat
from uncertainties import unumpy as unp
import formulas
from util import get_apl_ufloat
SAVE_PATH = "data/Ruby/ruby_data_new.csv"
n = 1.77

R = formulas.reflectivity(n)


OSCILLOSCOPE_TIME_ERROR = 0.0001e-3
OSCILLOSCOPE_VOLTAGE_PCT_ERROR = 0.03

In [54]:
# measured Transimission

I = get_apl_ufloat(43122.77)
I0 = get_apl_ufloat(50517.07)
I/I0, I, I0

(0.8536276945594825+/-2.6026704457596055e-07, 43122.77+/-0.01, 50517.07+/-0.01)

In [55]:
R

0.07727195714788411

In [56]:
T = (1-R)**2

In [57]:
T

0.8514270410656961

In [60]:
import pandas as pd
ruby_data = pd.read_csv("data/Ruby/ruby_data.csv")

ruby_data = ruby_data.dropna(axis =1 ,how = "all")



ruby_data.head()

,Wavelength,Ruby Intensity,Wavelength(nm),No Ruby Intensity,Transmission,Intensity_measurement_error
0,345.532,-25.20,345.532,-3.58,7.039106,0.01
1,345.749,-25.20,345.749,-3.58,7.039106,0.01
2,345.966,-25.20,345.966,-3.58,7.039106,0.01
3,346.184,-2.67,346.184,-3.58,0.745810,0.01
4,346.401,-12.91,346.401,-52.05,0.248031,0.01


In [61]:
ruby_data["Intensity_measurement_error"] = 0.01

In [62]:
from uncertainties import unumpy as unp


ruby_intensity = unp.uarray(list(ruby_data["Ruby Intensity"]),list(ruby_data["Intensity_measurement_error"]))
no_ruby_intensity = unp.uarray(list(ruby_data["No Ruby Intensity"]),list(ruby_data["Intensity_measurement_error"]))
ruby_data["Transmission_error"] = ruby_intensity/no_ruby_intensity

ruby_data.head()




,Wavelength,Ruby Intensity,Wavelength(nm),No Ruby Intensity,Transmission,Intensity_measurement_error,Transmission_error
0,345.532,-25.20,345.532,-3.58,7.039106,0.01,7.039+/-0.020
1,345.749,-25.20,345.749,-3.58,7.039106,0.01,7.039+/-0.020
2,345.966,-25.20,345.966,-3.58,7.039106,0.01,7.039+/-0.020
3,346.184,-2.67,346.184,-3.58,0.745810,0.01,0.7458+/-0.0035
4,346.401,-12.91,346.401,-52.05,0.248031,0.01,0.24803+/-0.00020


In [63]:
%load_ext autoreload
%autoreload 2
L = ufloat(2e-3, 0.05e-3)
def abs_coeff_ruby(T):

    return formulas.absorption_coefficent(T = T, R = R, L = L)


CUTOFF = 370
ruby_data = ruby_data[ruby_data["Wavelength"] >= CUTOFF]

print(min(ruby_data["Transmission_error"]), max(ruby_data["Transmission_error"]))

ruby_data["Absorption_Coefficient_error"] = ruby_data["Transmission_error"].apply(abs_coeff_ruby)
ruby_data["Absorption_Coeff"] = ruby_data["Absorption_Coefficient_error"].apply(lambda x: x.n)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.484428+/-0.000008 1.08565+/-0.00007


In [64]:
ruby_data.to_csv(SAVE_PATH,index = None)

In [65]:
ruby_data.head()

,Wavelength,Ruby Intensity,Wavelength(nm),No Ruby Intensity,Transmission,Intensity_measurement_error,Transmission_error,Absorption_Coefficient_error,Absorption_Coeff
113,370.018,195.30,370.018,226.47,0.862366,0.01,0.86237+/-0.00006,-6.38+/-0.16,-6.382909
114,370.234,185.74,370.234,240.81,0.771313,0.01,0.77131+/-0.00005,49.4+/-1.2,49.409465
115,370.450,174.82,370.450,265.38,0.658753,0.01,0.65875+/-0.00005,128.3+/-3.2,128.282211
116,370.666,197.34,370.666,258.56,0.763227,0.01,0.76323+/-0.00005,54.7+/-1.4,54.679090
117,370.882,176.86,370.882,247.63,0.714211,0.01,0.71421+/-0.00005,87.9+/-2.2,87.867886


In [66]:
absorption_maxima = max(ruby_data['Absorption_Coefficient_error'])
absorption_maxima

281.9729374708541+/-7.049328239062466

In [85]:
1/absorption_maxima, 1000*1/absorption_maxima

(0.0035464396298788927+/-8.866105114651161e-05,
 3.546439629878893+/-0.08866105114651159)

In [68]:
maxima2 = ruby_data[(ruby_data["Absorption_Coeff"] > 128.654) & (ruby_data["Absorption_Coeff"] < 128.655)]

In [69]:
maxima2["Absorption_Coefficient_error"].iloc[0],maxima2["Absorption_Coefficient_error"].iloc[0].n

(128.65459984347157+/-3.2163650035742855, 128.65459984347157)

In [70]:
maxima2["Absorption_Coefficient_error"].iloc[0].std_dev

3.2163650035742855

In [71]:
1/maxima2["Absorption_Coefficient_error"].iloc[0]

0.007772749681835366+/-0.00019431874249824596

In [72]:
maxima2["Wavelength"]

1006    557.954
Name: Wavelength, dtype: float64

In [73]:
from util import get_apl_ufloat

t2 = ufloat(0.008999999613, OSCILLOSCOPE_TIME_ERROR)
t1 = ufloat(0.0000399996643, OSCILLOSCOPE_TIME_ERROR)

In [74]:
t2-t1

0.0089599999487+/-1.414213562373095e-07

In [75]:
λ_green = get_apl_ufloat(532.270)
λ_abs_peak = get_apl_ufloat(414.690)
λ_ruby_red = get_apl_ufloat(693.899)

E_green = formulas.get_photon_energy_nm(λ_green)
E_abs_peak = formulas.get_photon_energy_nm(λ_abs_peak)
E_ruby_red = formulas.get_photon_energy_nm(λ_ruby_red)

E_green, E_abs_peak, E_ruby_red

(2.3287297799988727+/-4.3750911755290976e-05,
 2.9890110685090066+/-7.207820464706181e-05,
 1.7863017528487575+/-2.574296479529092e-06)

In [76]:
ufloat(200,1)/ ufloat(100, 1)


2.0+/-0.022360679774997897

In [77]:
(((1/200)**2 + (1/100)**2)**0.5) * 2

0.022360679774997897

In [78]:
formulas.pct_deviation(measured =I/I0, accepted = T)

'0.258466+/-0.000031%'

In [79]:
formulas.deviation(measured =I/I0, accepted = T)

'8455.367437593979'

In [80]:
formulas.get_deviations(measured =λ_ruby_red, accepted = 694.3)

('0.05776+/-0.00014%', '400.9999999999536')

CALCULATE ERROR IN FLUORESCENEC LIFETIME USING SUBSTITUE VALUES FOR V(t) and t. Necessary because we are using exponetntail decay parameter from excel, so exact values used in calculation are not known.

See lab report for formulas used in calcaulation.

In [81]:
import math
vt_v0 = ufloat(1,OSCILLOSCOPE_VOLTAGE_PCT_ERROR)/ufloat(math.e,OSCILLOSCOPE_VOLTAGE_PCT_ERROR)

vt_v0
t =ufloat(1/282.1, OSCILLOSCOPE_TIME_ERROR)

0.36787944117144233+/-0.01175949956047502

In [82]:
k = -unp.log(vt_v0)/t
k # aggrees with excel parameter

282.1+/-9.017508853619649

In [83]:
ed = 1/k
accepted_ed = 0.0035

ed,formulas.get_deviations(measured =ed, accepted = accepted_ed)

(0.0035448422545196734+/-0.00011331317410427575,
 ('1.3+/-3.2%', '0.3957373436420337'))

In [84]:
ed*1000

3.5448422545196734+/-0.11331317410427573